![./ImageLab.png](./Images/ImageLab.png "./ImageLab.png")

# Data Engineering with Lakeflow, Jobs, AutoLoader and more

## Load Data for Silver

In [0]:
# Read files from customers
df_bronze=spark.read \
  .table("medallion.bronze.dim_customer")

df_bronze.display()

In [0]:
# Treating data
from pyspark.sql import functions as F

# Treating customer_bk
df_bronze_treated = df_bronze \
  .withColumn("customer_bk", F.col('customer_bk').cast("string")) \
  .withColumn("customer_bk", F.regexp_replace(F.col('customer_bk'), "[^0-9]", ""))

# Treating customer_name
df_bronze_treated = df_bronze_treated \
  .withColumn("customer_name", F.trim(F.col('customer_name')))

# Treating birth_date
#Alternative 1 with coalesce
# formats = ["dd/MM/yyyy", "yyyy-MM-dd", "MM/dd/yyyy", "dd-MM-yyyy"]
# df_bronze_treated = df_bronze_treated.withColumn("birth_date",F.coalesce(*[F.try_to_date("birth_date", f) for f in formats]))

# Alternative 2 using ai function
df_bronze_treated = df_bronze_treated.selectExpr("* EXCEPT(birth_date)","ai_query('databricks-meta-llama-3-3-70b-instruct',concat('Padronize as datas para o formato 2025-01-01. Apenas me dê o resulado final e nada mais: ', birth_date)) as birth_date")

# Treating segment
df_bronze_treated = df_bronze_treated.withColumn("segment", F.trim(F.initcap(F.col('segment'))))

# Treating region
df_bronze_treated = df_bronze_treated.selectExpr("* EXCEPT(region)","ai_query('databricks-meta-llama-3-3-70b-instruct',concat('Padronize as regiões para o formato de sigla do estado do Brasil, como SP, MG, RJ, etc. Apenas me dê o resulado final e nada mais: ', region)) as region")

# Treating effective_ts
df_bronze_treated = df_bronze_treated.withColumn("effective_ts", F.date_format(F.trim(F.col('effective_ts')),"yyyy-MM-dd'T'HH:mm:ss'Z'"))

df_bronze_treated.display()

In [0]:
# Creating table using schema if it is first load
from pyspark.sql.window import Window
if not spark.catalog.tableExists("medallion.silver.dim_customer"):
    empty_df = df_bronze_treated

    w_last = Window.partitionBy("customer_bk").orderBy(F.col("effective_ts").desc())
    
    empty_df = empty_df \
    .withColumn("rn", F.row_number().over(w_last)) \
    .filter(F.col("rn") == 1) \
    .drop("rn")

    empty_df = empty_df \
        .withColumn("valid_from", F.col("effective_ts")) \
        .withColumn("valid_to",   F.lit(None).cast("timestamp")) \
        .withColumn("version", F.lit(1).cast("int")) \
        .withColumn("customer_sk", F.lit(None).cast("bigint")) \
        .withColumn("is_current", F.lit(True))




    from delta.tables import DeltaTable
    from pyspark.sql import functions as F
    from pyspark.sql.window import Window


    w_sk = Window.orderBy("customer_bk", "valid_from", "customer_name", "birth_date")


    empty_df = empty_df \
        .withColumn("customer_sk", F.row_number().over(w_sk).cast("bigint")) \
        .write.mode("overwrite") \
        .saveAsTable("medallion.silver.dim_customer")




# Incremental executions
else:
    from delta.tables import DeltaTable
    from pyspark.sql import functions as F
    from pyspark.sql.window import Window

    w_last = Window.partitionBy("customer_bk").orderBy(F.col("effective_ts").desc())

    df_bronze_dedup = (
        df_bronze_treated
        .select("customer_bk", "customer_name", "birth_date", "segment", "region", "effective_ts", "file_path", "file_mod_time") 
    .withColumn("rn", F.row_number().over(w_last))
    .filter(F.col("rn") == 1)
    .drop("rn")
)


    # Using SCD type 2
    delta = DeltaTable.forName(spark, "medallion.silver.dim_customer")
    delta.alias("delta").merge(source=df_bronze_dedup.alias("df"), condition="""
                                                        df.customer_bk = delta.customer_bk
                                                        """) \
    .whenMatchedUpdate(
        condition="""
            delta.customer_name <> df.customer_name OR
            delta.birth_date    <> df.birth_date    OR
            delta.segment       <> df.segment       OR
            delta.region        <> df.region
        """,
        set={
            "is_current": F.lit(False),
            "valid_to":   F.col("df.effective_ts"),
        }
    ) \
    .whenNotMatchedInsert(
        values={
            "customer_bk":   F.col("df.customer_bk"),
            "customer_name": F.col("df.customer_name"),
            "birth_date":    F.col("df.birth_date"),
            "segment":       F.col("df.segment"),
            "region":        F.col("df.region"),
            "valid_from":    F.col("df.effective_ts"),
            "valid_to":      F.lit(None).cast("timestamp"),
            "is_current":    F.lit(True),
            "version":       F.lit(1).cast("int"),
            "customer_sk":   F.lit(None).cast("bigint"),
            "file_path":     F.col("df.file_path"),
            "file_mod_time": F.col("df.file_mod_time")
        }
    ) \
    .execute()

    dim_customer=spark.read.table("medallion.silver.dim_customer")

    # Adding version
    version_null = dim_customer.filter(F.col("version").isNull())

    if version_null.count()>0:

        existing_with_ver = dim_customer.filter(F.col("version").isNotNull())

        max_ver_por_bk = existing_with_ver.groupBy("customer_bk") \
            .agg(F.max("version").alias("max_version"))

        version_null = version_null.join(max_ver_por_bk, on="customer_bk", how="left") \
            .withColumn("max_version", F.coalesce(F.col("max_version"), F.lit(0)))



        delta_version = DeltaTable.forName(spark, "medallion.silver.dim_customer")

        delta_version.alias("delta_version").merge(source=version_to_include.alias("version_to_include"), condition="""
        delta_version.customer_bk = version_to_include.customer_bk AND
        delta_version.valid_from  = version_to_include.valid_from AND
        delta_version.valid_to    <=> version_to_include.valid_to AND
        delta_version.is_current  = version_to_include.is_current
                                                            """) \
        .whenMatchedUpdate(
            set={
                "version": F.col("version_to_include.version")
            }
            ) \
        .execute()

    dim_customer=spark.read.table("medallion.silver.dim_customer")

    # Adding sk
    sk_null = dim_customer.filter(F.col("customer_sk").isNull())

    if sk_null.count()>0:
        max_sk = dim_customer.agg(F.max("customer_sk").alias("max_sk")).collect()[0]["max_sk"]
        if max_sk is None:
            max_sk = 0

        w = Window.orderBy("valid_from", "customer_bk")

        sk_to_include = sk_null.withColumn(
            "customer_sk",
            F.row_number().over(w) + F.lit(max_sk)
        )

        delta_sk = DeltaTable.forName(spark, "medallion.silver.dim_customer")

        delta_sk.alias("delta_sk").merge(source=sk_to_include.alias("sk_to_include"), condition="""
        delta_sk.customer_bk = sk_to_include.customer_bk AND
        delta_sk.valid_from  = sk_to_include.valid_from AND
        delta_sk.valid_to    <=> sk_to_include.valid_to AND
        delta_sk.is_current  = sk_to_include.is_current
                                                            """) \
        .whenMatchedUpdate(
            set={
                "customer_sk": F.col("sk_to_include.customer_sk")
            }
            ) \
        .execute()


    dim_customer = spark.read.table("medallion.silver.dim_customer")
    dim_current  = dim_customer.filter(F.col("is_current") == True)

    # Incremental join versus actual dimension
    df_join = df_bronze_dedup.alias("df").join(
        dim_current.alias("delta"),
        on=[F.col("df.customer_bk") == F.col("delta.customer_bk")],
        how="left"
    )

    # New BK (don't exist)
    df_new = df_join.filter(F.col("delta.customer_bk").isNull()) \
                    .select("df.*")

    # Existant BK with changes(already closed by whenMatchedUpdate)
    df_changed = df_join.filter(
        (F.col("delta.customer_bk").isNotNull()) &
        (
            (F.col("delta.customer_name") != F.col("df.customer_name")) |
            (F.col("delta.birth_date")    != F.col("df.birth_date"))    |
            (F.col("delta.segment")       != F.col("df.segment"))       |
            (F.col("delta.region")        != F.col("df.region"))
        )
    ).select("df.*")

    # Remaining rows that will be inserted in the table (new + new versions)
    df_remaining = df_new.unionByName(df_changed)

    # Adiciona colunas SCD: valid_from, valid_to, is_current
    df_remaining = (
        df_remaining
        .withColumn("valid_from", F.col("effective_ts"))
        .withColumn("valid_to",   F.lit(None).cast("timestamp"))
        .withColumn("is_current", F.lit(True))
    )

    # VERSION: next version by customer_bk
    max_ver_por_bk = dim_customer.groupBy("customer_bk") \
        .agg(F.max("version").alias("max_version"))

    df_remaining= df_remaining.join(
        max_ver_por_bk,
        on="customer_bk",
        how="left"
    ).withColumn(
        "max_version",
        F.coalesce(F.col("max_version"), F.lit(0))
    ).withColumn(
        "version",
        (F.col("max_version") + F.lit(1)).cast("int")
    ).drop("max_version")

    # CUSTOMER_SK: generate SK only for these rows
    max_sk = dim_customer.agg(F.max("customer_sk").alias("max_sk")).collect()[0]["max_sk"]
    if max_sk is None:
        max_sk = 0

    w_sk = Window.orderBy("valid_from", "customer_bk", "customer_name", "birth_date")

    df_remaining = df_remaining.withColumn(
        "customer_sk",
        (F.row_number().over(w_sk) + F.lit(max_sk)).cast("bigint")
    )

    # Now insert only the remaining rows
    delta = DeltaTable.forName(spark, "medallion.silver.dim_customer")

    (
        delta.alias("delta")
        .merge(
            source=df_remaining.alias("df"),
            condition="delta.customer_sk = df.customer_sk"
        )
        .whenNotMatchedInsert(
            values={
                "customer_sk":   F.col("df.customer_sk"),
                "customer_bk":   F.col("df.customer_bk"),
                "customer_name": F.col("df.customer_name"),
                "birth_date":    F.col("df.birth_date"),
                "segment":       F.col("df.segment"),
                "region":        F.col("df.region"),
                "valid_from":    F.col("df.valid_from"),
                "valid_to":      F.col("df.valid_to"),   # always NULL here
                "is_current":    F.col("df.is_current"), # always TRUE here
                "version":       F.col("df.version"),
                "file_path":     F.col("df.file_path"),
                "file_mod_time": F.col("df.file_mod_time")
            }
        )
        .execute()
    )

# Verify data
display(spark.read.table("medallion.silver.dim_customer"))